In [1]:
import torch
import torch.nn as nn  
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.metrics import classification_report
import ResNet as resnet
import DenseNet as densenet
import EfficientNet as efficientnet
import MobileNetV2 as mobilenet
import ViT as vit
import solver.solver as solver
import solver.solver_v2 as solver_v2
import solver.solver_v3 as solver_v3

/home/hyunjun/anaconda3/envs/test/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/hyunjun/anaconda3/envs/test/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZNK3c1010TensorImpl36is_contiguous_nondefault_policy_implENS_12MemoryFormatE
  warn(f"Failed to load image Python extension: {e}")


In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

n_epochs = 30
batch_size = 64
learning_rate = 1e-4
diff_drop = "v0"

model = efficientnet.efficientnet_b0(num_classes=8, diff_drop=diff_drop).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [3]:
# train_path = "/media/data1/hyunjun/cifar-10/train/"
# test_path = "/media/data1/hyunjun/cifar-10/test/"
train_path = "/media/data1/hyunjun/CelebA-HQ/dp-pix/m_10_eps_5/"
#test_path = "/media/data1/hyunjun/dp-cifar-10/dp-svd/k_8_eps_1/test/"

In [4]:
trans = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

celebAset = torchvision.datasets.ImageFolder(root=train_path, transform=trans)
# testset = torchvision.datasets.ImageFolder(root=test_path, transform=trans)
train_size = int(0.7 * len(celebAset))
trainset, testset = random_split(celebAset, [train_size, len(celebAset) - train_size])

train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, drop_last=False, )
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=True, drop_last=False, )

classes = celebAset.classes
print(classes)

['0', '1', '2', '3', '4', '5', '6', '7']


In [5]:
def train_loop(dataloader, model, criterion, optimizer, epoch):
    size = len(dataloader.dataset)
    total_correct = 0
    total_loss = 0
    
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        model.train()
        correct = 0
        
        pred = None
        p = 0.0
        if diff_drop == "v3":
            pred, p = model(X.to(device), epoch)
        else:
            pred, p = model(X.to(device))
        
        loss = criterion(pred, y.to(device))
        correct += (torch.argmax(pred, dim=1) == y.to(device)).type(torch.float).sum().item()
        total_correct += correct
        
        model.train()
        total_loss += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        
        if (batch % 50 == 0):
            print(f"Batch {batch:>5d}: Loss per batch is {loss.item():>7.6f} and Accuracy is {correct / batch_size:>7.6f}")
    print(f"For Iteration: Loss is {(total_loss / (size // batch_size)):>7.6f} and Accuracy is {total_correct / size:>7.6f}")

In [6]:
def test_loop(dataloader, model, criterion):
    test_loss = 0
    correct = 0

    model.eval()
    
    y_true = []
    y_pred = []
    
    with torch.no_grad():
        for X, y in dataloader:
            pred, _ = model(X.to(device))
            test_loss += criterion(pred, y.to(device)).item()
            
            pred = torch.argmax(pred, dim=1)
            correct += (pred == y.to(device)).type(torch.float).sum().item()
            
            pred = pred.cpu().numpy()
            y = y.cpu().numpy()
            y_true.extend(y)
            y_pred.extend(pred)
    
    print(classification_report(y_true=y_true, y_pred=y_pred, target_names=classes))
    
    return correct / len(dataloader.dataset)

In [7]:
max = 0.0
for epoch in range(n_epochs):
    print(f"Training: Epoch {epoch + 1} ------------------")
    train_loop(train_loader, model, criterion, optimizer, epoch + 1)
    print(f"----Test: Epoch {epoch + 1} ------------------")
    temp = test_loop(test_loader, model, criterion)

Training: Epoch 1 ------------------
Batch     0: Loss per batch is 2.073616 and Accuracy is 0.078125
Batch    50: Loss per batch is 1.794507 and Accuracy is 0.250000
Batch   100: Loss per batch is 1.725238 and Accuracy is 0.328125
Batch   150: Loss per batch is 1.746095 and Accuracy is 0.359375
Batch   200: Loss per batch is 1.757709 and Accuracy is 0.281250
Batch   250: Loss per batch is 1.956014 and Accuracy is 0.265625
Batch   300: Loss per batch is 1.706950 and Accuracy is 0.312500
For Iteration: Loss is 1.777008 and Accuracy is 0.316476
----Test: Epoch 1 ------------------


/home/hyunjun/anaconda3/envs/test/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hyunjun/anaconda3/envs/test/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hyunjun/anaconda3/envs/test/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

              precision    recall  f1-score   support

           0       0.47      0.41      0.44      2611
           1       0.00      0.00      0.00       191
           2       0.40      0.52      0.45      2399
           3       0.00      0.00      0.00       474
           4       0.26      0.71      0.38      1243
           5       0.33      0.05      0.09       757
           6       0.23      0.03      0.06       735
           7       0.25      0.00      0.00       590

    accuracy                           0.36      9000
   macro avg       0.24      0.22      0.18      9000
weighted avg       0.34      0.36      0.31      9000

Training: Epoch 2 ------------------
Batch     0: Loss per batch is 1.602400 and Accuracy is 0.406250
Batch    50: Loss per batch is 1.475239 and Accuracy is 0.406250
Batch   100: Loss per batch is 1.482392 and Accuracy is 0.390625
Batch   150: Loss per batch is 1.464289 and Accuracy is 0.421875
Batch   200: Loss per batch is 1.345423 and Accuracy

/home/hyunjun/anaconda3/envs/test/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hyunjun/anaconda3/envs/test/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hyunjun/anaconda3/envs/test/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

              precision    recall  f1-score   support

           0       0.45      0.79      0.57      2611
           1       0.00      0.00      0.00       191
           2       0.44      0.51      0.47      2399
           3       0.00      0.00      0.00       474
           4       0.37      0.28      0.32      1243
           5       0.31      0.10      0.16       757
           6       0.37      0.09      0.14       735
           7       0.29      0.13      0.18       590

    accuracy                           0.43      9000
   macro avg       0.28      0.24      0.23      9000
weighted avg       0.37      0.43      0.37      9000

Training: Epoch 3 ------------------
Batch     0: Loss per batch is 1.141579 and Accuracy is 0.578125
Batch    50: Loss per batch is 0.992787 and Accuracy is 0.656250
Batch   100: Loss per batch is 1.035488 and Accuracy is 0.625000
Batch   150: Loss per batch is 1.171012 and Accuracy is 0.515625
Batch   200: Loss per batch is 1.081322 and Accuracy

In [8]:
torch.save(model, './celebA_split_effib0_dppix_eps5.pt')